In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate
!pip -q install langchain

## HuggingFace

There are two Hugging Face LLM wrappers, one for a local pipeline and one for a model hosted on Hugging Face Hub. Note that these wrappers only work for models that support the following tasks: text2text-generation, text-generation


## Loading Alpaca7B

In [1]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

import torch

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

base_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    # load_in_8bit=True,
    device_map='auto',
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=4096,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [4]:
from langchain import PromptTemplate, LLMChain

template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
{instruction}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [5]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

question = "What is the capital of England?"

print(llm_chain.run(question))

/home/ubuntu/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1211: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


 The capital of England is London.


In [ ]:
question = "What are alpacas? and how are they different from llamas?"

print(llm_chain.run(question))

## Setting up a Chat with memory

In [6]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [7]:
# We are going to set the memory to go back 4 turns
window_memory = ConversationBufferWindowMemory(k=8)

In [8]:
conversation = ConversationChain(
    llm=local_llm, 
    verbose=True, 
    memory=window_memory
)

In [9]:
conversation.prompt.template

'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'

The following is a friendly conversation between a human and an AI called. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:

In [10]:
conversation.prompt.template = '''The following is a friendly conversation between a human and an AI called Alpaca. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. 

Current conversation:
{history}
Human: {input}
AI:'''

In [ ]:
conversation.predict(input="Hi there! I am Sam")

In [ ]:
conversation.predict(input="What is your name?")

In [ ]:
conversation.predict(input="Can you tell me what an Alpaca is?")

In [ ]:
conversation.predict(input="How is it different from a Llama?")

In [ ]:
conversation.predict(input="Can you give me some good names for a pet llama?")

In [ ]:
conversation.predict(input="Is your name Fred?")

In [ ]:
conversation.predict(input="What food should I feed my new llama?")

In [ ]:
conversation.predict(input="Do you remember my name?")

In [ ]:
conversation.predict(input="whats my name?")

In [11]:
conversation.predict(input="Hi there! I am DJ")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI called Alpaca. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. 

Current conversation:

Human: Hi there! I am DJ
AI:

> Finished chain.


" Hey there! It's nice to meet you too. What can I do for you?"

In [ ]:
conversation.predict(input="I want to know more about India")

In [ ]:
conversation.predict(input="Expalin ASC 606  in detail")

In [ ]:
conversation.predict(input="How do you multiply two vectors in pytorch?")

In [ ]:
conversation.predict(input="And how do you do it in Tensorflow?")

In [ ]:
conversation.predict(input="In Keras?")

In [ ]:
conversation.predict(input="Whats my name ?")

In [ ]:
conversation.predict(input="what is the best way to finetune you?")

In [12]:
conversation.predict(input="if i feed you pandas dataframe would you be able to make meaining out of the tabular data?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI called Alpaca. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. 

Current conversation:
Human: Hi there! I am DJ
AI:  Hey there! It's nice to meet you too. What can I do for you?
Human: if i feed you pandas dataframe would you be able to make meaining out of the tabular data?
AI:

> Finished chain.


' Sure, I think that should work. Can you provide the pandas dataframe so I can take a look at it?'

In [ ]:
conversation.predict(input="how shoud i feed you pandas dataframe, what format")